In [1]:
import torch
import transformers
from peft import LoraConfig, get_peft_model

from MERT_model import *
from MusicFM_model import *

mert = MERT_model()
mfm = MusicFM_model()

/home/z/miniconda3/envs/encoder_test_2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at m-a-p/MERT-v1-95M were not used when initializing MERTModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing MERTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MERTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MERTModel were not initialized from the model checkpoint at m-a-p/MERT-v1-95M

In [2]:
# sr = 24000
# length = 8
# dtype = torch.float32
# wav = (torch.rand((5, 2, sr * length)) - 0.5) * 2
# wav = wav.to("cuda:0")

# out_mert = mert(wav, sampling_rate=sr)
# print(out_mert.shape)
#out_mfm = mfm(wav, sampling_rate=sr)

In [19]:
def get_LoRA_trainable_modules(model, return_type_tuples = False):
    modules = [(n, type(m)) for n, m in model.named_modules()]
    trainable_modules = []
    for elem in modules:
        name, module_type = elem
        if module_type == torch.nn.Linear or module_type == torch.nn.Embedding or module_type == torch.nn.Conv2d or module_type == transformers.pytorch_utils.Conv1D:
            if return_type_tuples:
                trainable_modules.append((name, module_type))
            else:
                trainable_modules.append(name)
    return trainable_modules

get_LoRA_trainable_modules(MusicFM_model())

['musicfm.conv.conv.0.conv1',
 'musicfm.conv.conv.0.conv2',
 'musicfm.conv.conv.0.conv3',
 'musicfm.conv.conv.1.conv1',
 'musicfm.conv.conv.1.conv2',
 'musicfm.conv.conv.1.conv3',
 'musicfm.conv.linear',
 'musicfm.conformer.layers.0.ffn1.intermediate_dense',
 'musicfm.conformer.layers.0.ffn1.output_dense',
 'musicfm.conformer.layers.0.self_attn.linear_q',
 'musicfm.conformer.layers.0.self_attn.linear_k',
 'musicfm.conformer.layers.0.self_attn.linear_v',
 'musicfm.conformer.layers.0.self_attn.linear_out',
 'musicfm.conformer.layers.0.ffn2.intermediate_dense',
 'musicfm.conformer.layers.0.ffn2.output_dense',
 'musicfm.conformer.layers.1.ffn1.intermediate_dense',
 'musicfm.conformer.layers.1.ffn1.output_dense',
 'musicfm.conformer.layers.1.self_attn.linear_q',
 'musicfm.conformer.layers.1.self_attn.linear_k',
 'musicfm.conformer.layers.1.self_attn.linear_v',
 'musicfm.conformer.layers.1.self_attn.linear_out',
 'musicfm.conformer.layers.1.ffn2.intermediate_dense',
 'musicfm.conformer.layer

In [13]:
# l = [(n, type(m)) for n, m in MERT_model().named_modules()]
# for elem in l:
#     print(elem)

In [20]:
config = LoraConfig(
    target_modules=get_LoRA_trainable_modules(MusicFM_model()),
)
peft_model = get_peft_model(mfm, config)
peft_model.print_trainable_parameters()

trainable params: 3,119,920 || all params: 332,469,914 || trainable%: 0.9384


In [21]:
config = LoraConfig(
    target_modules=get_LoRA_trainable_modules(MERT_model()),
)
peft_model = get_peft_model(mert, config)
peft_model.print_trainable_parameters()

Some weights of the model checkpoint at m-a-p/MERT-v1-95M were not used when initializing MERTModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing MERTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MERTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MERTModel were not initialized from the model checkpoint at m-a-p/MERT-v1-95M and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,349,520 || all params: 96,002,121 || trainable%: 1.4057


In [2]:
mert.get_LoRA_model()

PeftModel(
  (base_model): LoraModel(
    (model): MERT_model(
      (model): MERTModel(
        (feature_extractor): HubertFeatureEncoder(
          (conv_layers): ModuleList(
            (0): HubertGroupNormConvLayer(
              (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
              (activation): GELUActivation()
              (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
            )
            (1-4): 4 x HubertNoLayerNormConvLayer(
              (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
              (activation): GELUActivation()
            )
            (5-6): 2 x HubertNoLayerNormConvLayer(
              (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
              (activation): GELUActivation()
            )
          )
        )
        (feature_projection): MERTFeatureProjection(
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (projection): lora.Li

In [3]:
mfm.get_LoRA_model()

PeftModel(
  (base_model): LoraModel(
    (model): MusicFM_model(
      (musicfm): MusicFM25Hz(
        (preprocessor_melspec_2048): MelSTFT(
          (mel_stft): MelSpectrogram(
            (spectrogram): Spectrogram()
            (mel_scale): MelScale()
          )
          (amplitude_to_db): AmplitudeToDB()
        )
        (quantizer_melspec_2048_0): RandomProjectionQuantizer()
        (conv): Conv2dSubsampling(
          (conv): Sequential(
            (0): Res2dModule(
              (conv1): lora.Conv2d(
                (base_layer): Conv2d(1, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Conv2d(1, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Conv2d(8, 512, kernel_size=(1, 1), stride=(1, 1), bias=